In [1]:
import pandas as pd
import altair as alt
import numpy as np
import os

In [2]:
data = pd.read_csv("data/vis2_data.csv")

In [3]:
def is_holiday(year, month, day, week_day):
    if month == 1 and day == 1:
        return 'Holiday'  # New Year's Day
    elif month == 1 and week_day == 0 and 15 <= day <= 21:
        return 'Holiday'  # Martin Luther King Jr. Day
    elif month == 2 and week_day == 0 and 15 <= day <= 21:
        return 'Holiday'  # Presidents' Day
    elif month == 5 and week_day == 0 and 25 <= day <= 31:
        return 'Holiday'  # Memorial Day
    elif month == 7 and day == 4:
        return 'Holiday'  # Independence Day
    elif month == 9 and week_day == 0 and 1 <= day <= 7:
        return 'Holiday'  # Labor Day
    elif month == 10 and week_day == 0 and 8 <= day <= 14:
        return 'Holiday'  # Columbus Day
    elif month == 11 and day == 11:
        return 'Holiday'  # Veterans Day
    elif month == 11 and week_day == 3 and 22 <= day <= 28:
        return 'Holiday'  # Thanksgiving Day
    elif month == 12 and day == 25:
        return 'Holiday'  # Christmas Day
    elif week_day in [6, 7]:
        return 'Holiday'
    else:
        return 'Workday'

data['Day Type'] = data.apply(lambda row: is_holiday(row['Year'], row['Month'], row['Date'], row['Week Day']), axis=1)

data['Count'] = data.groupby(['Year', 'Month']).transform('size')
heatmap_data = data[['Year', 'Month', 'Count', 'Day Type']].drop_duplicates()

month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

year_brush = alt.selection_interval(fields=['Year'], encodings=['y'])

heatmap = alt.Chart(heatmap_data).mark_rect(
    stroke='white',
    strokeWidth=5
).encode(
    x=alt.X(
        'Month:O',
        title='Month',
        sort=list(range(1, 13)),
        axis=alt.Axis(
            labels=True,
            labelAngle=0,
            ticks=False,
            domain=False,
            labelFontSize=14,
            titleFontSize=14,
            labelExpr='datum.value == 1 ? "Jan" : datum.value == 2 ? "Feb" : '
                      'datum.value == 3 ? "Mar" : datum.value == 4 ? "Apr" : '
                      'datum.value == 5 ? "May" : datum.value == 6 ? "Jun" : '
                      'datum.value == 7 ? "Jul" : datum.value == 8 ? "Aug" : '
                      'datum.value == 9 ? "Sep" : datum.value == 10 ? "Oct" : '
                      'datum.value == 11 ? "Nov" : "Dec"'
        )
    ),
    y=alt.Y(
        'Year:O',
        title='Year',
        sort='descending',
        axis=alt.Axis(
            ticks=False,
            domain=False,
            labelFontSize=14,
            titleFontSize=14
        )
    ),
    color=alt.condition(
        year_brush,
        alt.Color(
            'Count:Q',
            scale=alt.Scale(domain=[heatmap_data['Count'].min(), heatmap_data['Count'].max()],
                            range=['#7F92B1', '#BA0C2F']),
            legend=alt.Legend(title="Event Count", titleFontSize=14, labelFontSize=14)
        ),
        alt.value('lightgray') 
    ),
    tooltip=['Year', 'Month', 'Count', 'Day Type']
).add_selection(
    year_brush
).properties(
    width=400,
    height=350,
    title=alt.TitleParams(
        text="Mass Shooting Events by Year and Month",
        fontSize=20
    )
)

stacked_data = data.copy()

stacked_summary = stacked_data.groupby(['Year', 'Day Type']).agg(
    Count=('Week Day', 'count'),
    Victims_Killed=('Victims Killed', 'sum'),
    Victims_Injured=('Victims Injured', 'sum')
).reset_index()

stacked_chart = alt.Chart(stacked_summary).transform_fold(
    ['Count', 'Victims_Killed', 'Victims_Injured'],
    as_=['OriginalCategory', 'Value']
).transform_calculate(
    Category=alt.expr.replace(alt.datum.OriginalCategory, '_', ' ')
).mark_bar().encode(
    x=alt.X(
        'Category:N', 
        title=None, 
        axis=alt.Axis(labelAngle=0, labelFontSize=14, titleFontSize=14)  
    ),
    y=alt.Y(
        'Value:Q', 
        stack='normalize', 
        title='Proportion', 
        axis=alt.Axis(grid=False, labelFontSize=14, titleFontSize=14)  
    ), 
    color=alt.Color(
        'Day Type:N',
        scale=alt.Scale(domain=['Holiday', 'Workday'], range=['#BA0C2F', '#0A3161']),
        legend=alt.Legend(title="Day Type", titleFontSize=14, labelFontSize=14) 
    ),
    tooltip=['Year:N', 'Day Type:N', 'Category:N', 'Value:Q']
).transform_filter(
    year_brush
).properties(
    width=400,
    height=350,
    title=alt.TitleParams(
        text="Proportion of Events and Victims by Day Type",
        fontSize=20
    )
)

final_chart = alt.hconcat(
    heatmap,
    stacked_chart
).resolve_legend(
    color="independent"
).configure_view(
    strokeWidth=0
)

final_chart

C:\Users\Raymone\AppData\Local\Temp\ipykernel_26620\992683429.py:81: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.HConcatChart(...)

In [5]:
# save the chart
final_chart.save('Viz2.html')

In [ ]:
temp_file = 'mass_shooting_season_holiday_chart.html'

custom_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Temporal Analysis of Mass Shooting Incidents in the U.S.</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            line-height: 1.6;
            margin: 40px;
        }
        h1 {
            color: #333;
            text-align: center;
        }
        p {
            color: #555;
        }
        .reference {
            font-size: 0.9em;
            color: #888;
            margin-top: 20px;
        }
        .chart-container {
            display: flex;
            justify-content: center;
            align-items: center;
            margin-top: 20px;
            width: 100%;
            text-align: center;
        }
        iframe {
            display: block;
            margin: 0 auto;
        }
    </style>
</head>
<body>
    <h1>Temporal Analysis of Mass Shooting Incidents in the U.S.</h1>
    <p>
        If organizing mass shooting data in another way, i.e. by aggregating events by year and month, it is worth noticing that the frequency of mass shootings is significantly higher in the second and third quarters of the year. This trend is particularly noticeable during the hottest months. Since 2020, the number of mass shootings in summer (June, July, August) has averaged around 70 per month, peaking at 94 in June 2020, whereas in winter (December, January, February), monthly incidents were around 30.
    </p>
    <p>
        The seasonal pattern of mass shootings may be influenced by climatic factors: rising temperatures can increase stress hormones in the nervous system, thereby heightening individual violent tendencies. Additionally, warmer temperatures lead to more social interaction and outdoor activities, which can elevate the risk of interpersonal conflicts<sup>1</sup>. Numerous studies have shown a positive correlation between rising temperatures and increased rates of firearm violence<sup>2</sup>.
    </p>
    <p>
        Besides the significant seasonal pattern, mass shootings are also closely linked to holidays. Over 50% of mass shootings in the U.S. occur during weekends and public holidays, accounting for more than half of the casualties as well. In contrast, weekends and holidays make up less than 30% of the days in a year. Increased crowd gatherings and the dispersal of law enforcement resources may be contributing factors.
    </p>
"""

reference_html = """
    <div class="reference">
        <strong>References:</strong><br>
        1. Lyons VH, Gause EL, Spangler KR, Wellenius GA, Jay J. Analysis of Daily Ambient Temperature and Firearm Violence in 100 US Cities. JAMA Netw Open. 2022;5(12):e2247207.<br>
        2. Kieltyka J, Kucybala K, Crandall M. Ecologic factors relating to firearm injuries and gun violence in Chicago. J Forensic Leg Med. 2016;37:87-90. doi:10.1016/j.jflm.2015.11.003.
    </div>
"""

# with open(temp_file, 'r', encoding='utf-8') as file:
#     chart_html = file.read()

# final_html = f"""
# {custom_html}
# <div class="chart-container">
#     {chart_html}
# </div>
# {reference_html}
# </body>
# </html>
# """

# output_file = 'mass shooting seanson and holiday analysis.html'
# with open(output_file, 'w', encoding='utf-8') as file:
#     file.write(final_html)

# print(f"HTML file with references added saved as {output_file}")

FileNotFoundError: [Errno 2] No such file or directory: 'mass_shooting_season_holiday_chart.html'